In [ ]:
#token processing was conducted in batches across different VM isntances to increase speed. 

In [ ]:
import pandas as pd

In [ ]:
from transformers import BertTokenizer

In [ ]:
import pickle

In [ ]:
from datetime import datetime

In [ ]:
step=1000

In [ ]:
sentence_max = int(1.331956e+05)

In [ ]:
def tokenize(sentences, batch, nrows):# Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    LOG_EVERY_N = 100
    counting = 0
    # For every sentence...
    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 264,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

        if (counting % LOG_EVERY_N) == 0:
            print (f'logging: ...{str(counting)} sentences at {datetime.now()}')
        counting+=1

    file_out = open(f'input_ids_token_processing_sentences_{str(batch)}_{str(batch+nrows)}.pk', "wb")
    pickle.dump(input_ids, file_out)
    file_out = open(f'attention_masks_token_processing_sentences_{str(batch)}_{str(batch+nrows)}.pk', "wb")
    pickle.dump(attention_masks, file_out)

In [ ]:
# Store the model we want to use
MODEL_NAME = "bert-base-uncased"

In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=True)

In [ ]:
dftest = pd.read_csv('final_concat_df.csv',skiprows = range(1,61000)) 

In [ ]:
dftest.tail()

In [ ]:
dftest.shape

In [ ]:
dftest2 = pd.read_csv('final_concat_df.csv',skiprows = range(1,61000), nrows = 134) 

In [ ]:
dftest2.tail()

In [ ]:
dftest2.shape

In [ ]:
for x in range(40000, 61134, step):
    print (f'batch starting row {x}')
    if x == 61000:
        nrows= 61134 - 61000 + 1
    else:
        nrows= step
    df = pd.read_csv('final_concat_df.csv',skiprows = range(1,x), nrows = nrows) 
    sentences = df['TEXT'].apply(lambda x: x[:sentence_max]  if len(x) > sentence_max else x).values
    tokenize(sentences, x, nrows)

In [ ]:
for x in range(61000, 61134, step):
    print (f'batch starting row {x}')
    if x == 61000:
        nrows= 61134 - 61000 + 1
    else:
        nrows= step
    df = pd.read_csv('final_concat_df.csv',skiprows = range(1,x), nrows = nrows) 
    sentences = df['TEXT'].apply(lambda x: x[:sentence_max]  if len(x) > sentence_max else x).values
    tokenize(sentences, x, nrows)

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
df.shape